In [1]:
import os
os.environ["ETS_TOOLKIT"] = "null"
os.environ["QT_QPA_PLATFORM"] = "offscreen"

from mayavi import mlab

#all imports
import carla #the sim library itself
import glob
import os
import sys 
import time
import threading 
from mayavi import mlab


ModuleNotFoundError: No module named 'vtkCommonCorePython'

In [2]:
# # connect to the sim 
# client = carla.Client('localhost', 2000)

try:
    # Connect to the server on localhost at port 2000
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)  # seconds

    # Retrieve the world
    world = client.get_world()
    print("Connected to CARLA world:", world.get_map().name)

except Exception as e:
    print("Failed to connect to CARLA:", e)


Connected to CARLA world: Carla/Maps/Town10HD_Opt


In [4]:
actor_list = []

def generate_lidar_blueprint(blueprint_library):
    #Define semantic lidar sensor here
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast_semantic')
    lidar_blueprint.set_attribute('channels', str(64))
    lidar_blueprint.set_attribute('points_per_second', str(56000*10))
    lidar_blueprint.set_attribute('rotation_frequency', str(100))
    lidar_blueprint.set_attribute('range', str(100.0*200))
    lidar_blueprint.set_attribute('upper_fov', str(15))
    lidar_blueprint.set_attribute('lower_fov', str(-25))
    
    return lidar_blueprint

def semantic_lidar_data(point_cloud, lidar_point_cloud_buffer):
    """Prepares a point cloud with semantic segmentation colors"""
    matrix_representational_data = np.frombuffer(point_cloud.raw_data, dtype=np.dtype([
        ('x', np.float32), ('y', np.float32), ('z', np.float32),
        ('CosAngle', np.float32), ('ObjIdx', np.uint32), ('ObjTag', np.uint32)]))
    lidar_points = np.array([matrix_representational_data['x'], matrix_representational_data['y'], matrix_representational_data['z']])
    
    print("\n matrix_representational_data \n",matrix_representational_data)
    
    #define object tag from matrix_representational_data and store in labels variable
    labels = np.array(matrix_representational_data['ObjTag'], dtype=np.float32)
    #store lidar points into lidar_point_cloud_buffer['pts']
    lidar_point_cloud_buffer['pts'] = lidar_points
    #store lables into lidar_point_cloud_buffer['intensity']
    lidar_point_cloud_buffer['intensity'] = labels

def carlaThreadingLoop(world):
    frame = 0
    while True:
        time.sleep(0.005)
        world.tick()
        frame += 1

In [8]:
import carla
import numpy as np
import open3d as o3d
import time
import threading
import os

# === Connect to CARLA ===
try:
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()
    print("Connected to CARLA world:", world.get_map().name)
except Exception as e:
    print("Failed to connect to CARLA:", e)
    raise

actor_list = []

# === LiDAR blueprint ===
def generate_lidar_blueprint(blueprint_library):
    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast_semantic')
    lidar_bp.set_attribute('channels', '64')
    lidar_bp.set_attribute('points_per_second', str(56000*10))
    lidar_bp.set_attribute('rotation_frequency', '100')
    lidar_bp.set_attribute('range', '2000')  # reduce for visualization
    lidar_bp.set_attribute('upper_fov', '15')
    lidar_bp.set_attribute('lower_fov', '-25')
    return lidar_bp

# === Process semantic LiDAR data ===
def semantic_lidar_data(point_cloud, lidar_buffer):
    data = np.frombuffer(point_cloud.raw_data, dtype=np.dtype([
        ('x', np.float32), ('y', np.float32), ('z', np.float32),
        ('CosAngle', np.float32), ('ObjIdx', np.uint32), ('ObjTag', np.uint32)
    ]))
    pts = np.vstack((data['x'], data['y'], data['z'])).T
    labels = data['ObjTag'].astype(np.float32)
    lidar_buffer['pts'] = pts
    lidar_buffer['intensity'] = labels

# === CARLA tick thread ===
def carla_thread(world):
    while True:
        world.tick()
        time.sleep(0.005)

# === Spawn vehicle ===
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('model3')[0]
spawn_point = world.get_map().get_spawn_points()[1]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
vehicle.set_autopilot()
actor_list.append(vehicle)

# === Attach LiDAR ===
lidar_bp = generate_lidar_blueprint(blueprint_library)
lidar_transform = carla.Transform(carla.Location(x=0, y=0, z=2.0))
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)
actor_list.append(lidar)

# === Open3D visualizer ===
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="CARLA LiDAR", width=960, height=540)
pcd = o3d.geometry.PointCloud()
vis.add_geometry(pcd)

# === Prepare output folder ===
output_folder = r"D:\KP_Alexis_Doci\lidar_alex"
os.makedirs(output_folder, exist_ok=True)
frame_idx = 0

# === Shared buffer for LiDAR ===
lidar_buffer = {'pts': np.zeros((1,3)), 'intensity': np.zeros(1)}
lidar.listen(lambda data: semantic_lidar_data(data, lidar_buffer))

# === Start CARLA tick thread ===
threading.Thread(target=carla_thread, args=(world,), daemon=True).start()

# === Visualization loop with saving frames ===
try:
    while True:
        pts = lidar_buffer['pts']
        intensity = lidar_buffer['intensity']

        if pts.shape[0] > 0 and intensity.shape[0] > 0:
            # Trim arrays to match
            min_len = min(pts.shape[0], intensity.shape[0])
            pts_trim = pts[:min_len]
            intensity_trim = intensity[:min_len]

            # Center the point cloud for visibility
            pts_centered = pts_trim - np.mean(pts_trim, axis=0)
            pcd.points = o3d.utility.Vector3dVector(pts_centered)

            # Colorize points
            colors = np.zeros((min_len, 3))
            colors[:,0] = np.clip(intensity_trim/255.0, 0, 1)  # red tint
            pcd.colors = o3d.utility.Vector3dVector(colors)

            vis.update_geometry(pcd)

            # Set camera view once
            ctr = vis.get_view_control()
            ctr.set_front([0.5, -1, -0.3])
            ctr.set_lookat([0,0,0])
            ctr.set_up([0,0,1])
            ctr.set_zoom(0.4)

            # Save frame
            filename = os.path.join(output_folder, f"frame_{frame_idx:04d}.png")
            vis.capture_screen_image(filename)
            frame_idx += 1

        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.05)

except KeyboardInterrupt:
    print("Exiting...")

finally:
    print("Destroying actors...")
    for actor in actor_list:
        actor.destroy()
    vis.destroy_window()
    print("Done.")


NameError: name 'mlab' is not defined

In [2]:
#!/usr/bin/env python

# Copyright (c) 2020 Computer Vision Center (CVC) at the Universitat Autonoma de
# Barcelona (UAB).
#
# This work is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.

"""
Lidar projection on RGB camera example
"""

import glob
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import argparse
from queue import Queue
from queue import Empty
from matplotlib import cm

try:
    import numpy as np
except ImportError:
    raise RuntimeError('cannot import numpy, make sure numpy package is installed')

try:
    from PIL import Image
except ImportError:
    raise RuntimeError('cannot import PIL, make sure "Pillow" package is installed')

VIRIDIS = np.array(cm.get_cmap('viridis').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

def sensor_callback(data, queue):
    """
    This simple callback just stores the data on a thread safe Python Queue
    to be retrieved from the "main thread".
    """
    queue.put(data)


def tutorial(args):
    """
    This function is intended to be a tutorial on how to retrieve data in a
    synchronous way, and project 3D points from a lidar to a 2D camera.
    """
    # Connect to the server
    client = carla.Client(args.host, args.port)
    client.set_timeout(10.0)
    world = client.get_world()
    bp_lib = world.get_blueprint_library()

    traffic_manager = client.get_trafficmanager(8000)
    traffic_manager.set_synchronous_mode(True)

    original_settings = world.get_settings()
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 3.0
    world.apply_settings(settings)

    vehicle = None
    camera = None
    lidar = None

    try:
        if not os.path.isdir('_out'):
            os.mkdir('_out')
        # Search the desired blueprints
        vehicle_bp = bp_lib.filter("vehicle.lincoln.mkz_2017")[0]
        camera_bp = bp_lib.filter("sensor.camera.rgb")[0]
        lidar_bp = bp_lib.filter("sensor.lidar.ray_cast")[0]

        # Configure the blueprints
        camera_bp.set_attribute("image_size_x", str(args.width))
        camera_bp.set_attribute("image_size_y", str(args.height))

        if args.no_noise:
            lidar_bp.set_attribute('dropoff_general_rate', '0.0')
            lidar_bp.set_attribute('dropoff_intensity_limit', '1.0')
            lidar_bp.set_attribute('dropoff_zero_intensity', '0.0')
        lidar_bp.set_attribute('upper_fov', str(args.upper_fov))
        lidar_bp.set_attribute('lower_fov', str(args.lower_fov))
        lidar_bp.set_attribute('channels', str(args.channels))
        lidar_bp.set_attribute('range', str(args.range))
        lidar_bp.set_attribute('points_per_second', str(args.points_per_second))

        # Spawn the blueprints
        vehicle = world.spawn_actor(
            blueprint=vehicle_bp,
            transform=world.get_map().get_spawn_points()[0])
        vehicle.set_autopilot(True)
        camera = world.spawn_actor(
            blueprint=camera_bp,
            transform=carla.Transform(carla.Location(x=1.6, z=1.6)),
            attach_to=vehicle)
        lidar = world.spawn_actor(
            blueprint=lidar_bp,
            transform=carla.Transform(carla.Location(x=1.0, z=1.8)),
            attach_to=vehicle)

        # Build the K projection matrix:
        # K = [[Fx,  0, image_w/2],
        #      [ 0, Fy, image_h/2],
        #      [ 0,  0,         1]]
        image_w = camera_bp.get_attribute("image_size_x").as_int()
        image_h = camera_bp.get_attribute("image_size_y").as_int()
        fov = camera_bp.get_attribute("fov").as_float()
        focal = image_w / (2.0 * np.tan(fov * np.pi / 360.0))

        # In this case Fx and Fy are the same since the pixel aspect
        # ratio is 1
        K = np.identity(3)
        K[0, 0] = K[1, 1] = focal
        K[0, 2] = image_w / 2.0
        K[1, 2] = image_h / 2.0

        # The sensor data will be saved in thread-safe Queues
        image_queue = Queue()
        lidar_queue = Queue()

        camera.listen(lambda data: sensor_callback(data, image_queue))
        lidar.listen(lambda data: sensor_callback(data, lidar_queue))

        for frame in range(args.frames):
            world.tick()  # advance the simulation
            world_frame = world.get_snapshot().frame

            try:
                # Get synchronized data
                image_data = image_queue.get(True, 1.0)
                lidar_data = lidar_queue.get(True, 1.0)

                if lidar_data is None or image_data is None:
                    print(f"[Warning] Missing data at frame {frame}")
                    continue

            except Empty:
                print(f"[Warning] Some sensor data missed at frame {frame}")
                continue

            # Convert image to NumPy array
            im_array = np.frombuffer(image_data.raw_data, dtype=np.uint8)
            im_array = np.reshape(im_array, (image_data.height, image_data.width, 4))
            im_array = im_array[:, :, :3][:, :, ::-1]  # BGRA -> RGB

            # Convert Lidar data to NumPy array safely
            points = np.frombuffer(lidar_data.raw_data, dtype=np.float32)
            points = np.reshape(points, (-1, 4))  # (N, 4) -> x, y, z, intensity
            xyz = points[:, :3].T  # shape (3, N)
            intensity = points[:, 3]

            # Add homogeneous coordinate for transformations
            xyz_h = np.vstack([xyz, np.ones((1, xyz.shape[1]))])  # (4, N)

            # Lidar -> World
            lidar_2_world = lidar.get_transform().get_matrix()
            world_points = np.dot(lidar_2_world, xyz_h)

            # World -> Camera
            world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
            sensor_points = np.dot(world_2_camera, world_points)

            # UE4 -> OpenCV camera coordinates
            point_in_camera_coords = np.array([
                sensor_points[1],
                -sensor_points[2],
                sensor_points[0]
            ])

            # Project to 2D using K
            points_2d = np.dot(K, point_in_camera_coords)
            points_2d /= points_2d[2, :]  # normalize

            # Filter points inside the image frame and in front of camera
            u = points_2d[0, :].astype(int)
            v = points_2d[1, :].astype(int)
            mask = (u >= 0) & (u < image_w) & (v >= 0) & (v < image_h) & (points_2d[2, :] > 0)
            u, v, intensity = u[mask], v[mask], intensity[mask]

            # Map intensity to color
            intensity = np.clip(4 * intensity - 3, 0, 1)
            color_map = np.array([
                np.interp(intensity, VID_RANGE, VIRIDIS[:, 0]) * 255,
                np.interp(intensity, VID_RANGE, VIRIDIS[:, 1]) * 255,
                np.interp(intensity, VID_RANGE, VIRIDIS[:, 2]) * 255
            ]).astype(np.uint8).T

            # Draw points
            if args.dot_extent <= 0:
                im_array[v, u] = color_map
            else:
                for i in range(len(u)):
                    im_array[
                        max(0, v[i]-args.dot_extent):min(image_h, v[i]+args.dot_extent),
                        max(0, u[i]-args.dot_extent):min(image_w, u[i]+args.dot_extent)
                    ] = color_map[i]

            # Save image
            image = Image.fromarray(im_array)
            image.save(f"_out/{image_data.frame:08d}.png")

            sys.stdout.write(f"\rFrame {frame}/{args.frames} processed ")
            sys.stdout.flush()


    finally:
        # Apply the original settings when exiting.
        world.apply_settings(original_settings)

        # Destroy the actors in the scene.
        if camera:
            camera.destroy()
        if lidar:
            lidar.destroy()
        if vehicle:
            vehicle.destroy()


def main():
    """Start function"""
    argparser = argparse.ArgumentParser(
        description='CARLA Sensor sync and projection tutorial')
    argparser.add_argument(
        '--host',
        metavar='H',
        default='127.0.0.1',
        help='IP of the host server (default: 127.0.0.1)')
    argparser.add_argument(
        '-p', '--port',
        metavar='P',
        default=2000,
        type=int,
        help='TCP port to listen to (default: 2000)')
    argparser.add_argument(
        '--res',
        metavar='WIDTHxHEIGHT',
        default='640x480',
        help='window resolution (default: 1280x720)')
    argparser.add_argument(
        '-f', '--frames',
        metavar='N',
        default=500,
        type=int,
        help='number of frames to record (default: 500)')
    argparser.add_argument(
        '-d', '--dot-extent',
        metavar='SIZE',
        default=2,
        type=int,
        help='visualization dot extent in pixels (Recomended [1-4]) (default: 2)')
    argparser.add_argument(
        '--no-noise',
        action='store_true',
        help='remove the drop off and noise from the normal (non-semantic) lidar')
    argparser.add_argument(
        '--upper-fov',
        metavar='F',
        default=30.0,
        type=float,
        help='lidar\'s upper field of view in degrees (default: 15.0)')
    argparser.add_argument(
        '--lower-fov',
        metavar='F',
        default=-25.0,
        type=float,
        help='lidar\'s lower field of view in degrees (default: -25.0)')
    argparser.add_argument(
        '-c', '--channels',
        metavar='C',
        default=64.0,
        type=float,
        help='lidar\'s channel count (default: 64)')
    argparser.add_argument(
        '-r', '--range',
        metavar='R',
        default=100.0,
        type=float,
        help='lidar\'s maximum range in meters (default: 100.0)')
    argparser.add_argument(
        '--points-per-second',
        metavar='N',
        default='100000',
        type=int,
        help='lidar points per second (default: 100000)')
    args = argparser.parse_args()
    args.width, args.height = [int(x) for x in args.res.split('x')]
    args.dot_extent -= 1

    try:
        tutorial(args)

    except KeyboardInterrupt:
        print('\nCancelled by user. Bye!')


if __name__ == '__main__':

    main()

usage: ipykernel_launcher.py [-h] [--host H] [-p P] [--res WIDTHxHEIGHT]
                             [-f N] [-d SIZE] [--no-noise] [--upper-fov F]
                             [--lower-fov F] [-c C] [-r R]
                             [--points-per-second N]
ipykernel_launcher.py: error: argument -f/--frames: invalid int value: 'C:\\Users\\admin\\AppData\\Roaming\\jupyter\\runtime\\kernel-9da9a21f-f131-4f29-8f09-7afe31d75a7a.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\envs\carla-sim\lib\argparse.py", line 2409, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: 'C:\\Users\\admin\\AppData\\Roaming\\jupyter\\runtime\\kernel-9da9a21f-f131-4f29-8f09-7afe31d75a7a.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\envs\carla-sim\lib\argparse.py", line 1787, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Users\admin\anaconda3\envs\carla-sim\lib\argparse.py", line 1993, in _parse_known_args
    start_index = consume_optional(start_index)
  File "C:\Users\admin\anaconda3\envs\carla-sim\lib\argparse.py", line 1933, in consume_optional
    take_action(action, args, option_string)
  File "C:\Users\admin\anaconda3\envs\carla-sim\lib\argparse.py", line 1845, in take_action
    argument_values = self._get_values

TypeError: object of type 'NoneType' has no len()